In [ ]:
from random import randint

f1 = open('/Users/enn/Desktop/sml/assignment1/data/train.txt', 'r')

xdata = []

x1 = []
x2 = []
y1 = []
y2 = []
ydata = []
k = []
n = []
l = []

for lines in f1.readlines():
    lines = lines.replace('\n', '')
    item = lines.replace('\t', ' ').split(' ')
    l.append(item)

    for i in range(len(item)):
        if (i > 0):
            x1.append([item[0], item[i]])
            y1.append(1)
m = l  
for i in l:
    for j in m:
        k = [x for x in i if x in j]
        if k == []:
            n.append([i[0], j[0]])
            
for p in range(len(n)):
    if p % 300 == 0:
        x2.append(n[p])
        y2.append(0)
#print(len(x2))
#print(x2)
xdata = x1 + x2
print(xdata)
ydata = y1 + y2
print(len(ydata))

f1.close()

In [ ]:
import numpy as np
f2 = open('/Users/enn/Desktop/sml/assignment1/data/test-public.csv', 'r')
test = f2.readlines()
testx = []
#print(test)
for line in test[1:]:
    line = line.strip('\n')
    linelist = line.split(',')
    testx.append(linelist[1:])
xtest = []
for item in testx:
    #print(item)
    item = list(map(float, item))
    #print(item)
    xtest.append(item)
xtest = np.array(xtest)
print(xtest)
f2.close()

In [ ]:
from random import choice
import numpy as np
import pandas as pd
import networkx as nx
import json

train_data = '/Users/enn/Desktop/sml/assignment1/data/train.txt'
test_data = '/Users/enn/Desktop/sml/assignment1/data/test-public.csv'
feature = open('/Users/enn/Desktop/sml/assignment1/data/nodes.json', 'r')

feature_file= json.load(feature)
#print(feature_file)
for dic in feature_file:
    if dic['id'] == 0:
        print(dic)

def get_data(input_file, negative_padding=False):
    data = {}
    relations = 0
    nodes = []
    with open(input_file, 'r') as f:
        for line in f.readlines():
            line = line.replace('\n', '')
            line = line.replace('\t', ' ').split(' ')
            data.setdefault(line[0], line[1:])
            relations += len(line[1:])
            nodes.extend([line[0]])
            nodes.extend(line[1:])
            #break
    nodes = list(set(nodes))
    #print(data)
    #print(nodes)
    #print(relations)
    if negative_padding:
        neg = {}
        for _ in range(relations):
            source = choice(nodes)
            target = choice(nodes)
            while target == source:
                target = choice(nodes)
            try:
                while target in data[source]:
                    target = choice(nodes)
            except KeyError:
                pass
            neg.setdefault(source, [])
            neg[source] = neg[source] + [target]
        return data, neg
    return data
train_data, neg_train_data = get_data(train_data, negative_padding=True)
#print(neg_train_data)

test_data = {}

with open(test_data, 'r') as f:
    for line in f.readlines()[1:]:
        #print(line)
        line = line.strip('\n')
        line = line.split(',') 
        if line[1] in test_data.keys():
            test_data[line[1]] += line[2:]
        else:
            test_data.setdefault(line[1], line[2:])
#print(test_data)

In [ ]:
g = nx.Graph(train_data)
print(g.nodes())

In [6]:
def graph_construct(data):
    graph = nx.Graph(data)
    digraph = nx.DiGraph(data)
    return graph, digraph

In [7]:
def to_preprocess(data_list, negative = False):
    graph, digraph = graph_construct(data_list[0])
    
    pagerank = nx.pagerank(digraph)
    auth = nx.hits(digraph)
    
    def feature_extraction(data, negative):
        counter = 0
        pre_feature = []
        label = []
        print(len(data))
        for source_node, tar_list in data.items():
            for target_node in tar_list:
                pre_feature.append(get_feature(source_node, target_node))
                label.append(0) if negative else label.append(1)
                counter += 1
                if counter % 100 == 0:
                    print('Instance processing over {} out of {}'.format(counter, nx.number_of_edges(graph)))
                if counter == 53697:
                    assert len(pre_feature) == len(label)
                    return pre_feature, label
                    #return label
        assert len(pre_feature) == len(label)
        return pre_feature, label
        #return label

    def get_feature(source, target):

        features = {}

        def set_feature(key, val):
            if key not in features:
                features[key] = val
                
        try:
            #for dic in feature_file:
             #   if dic['id'] == source:
              #      set_feature('source_feature', dic)
               # elif dic['id'] == target:
                #    set_feature('target_feature', dic)
            source_succ = set(digraph.successors(source))
            source_pred = set(digraph.predecessors(source))
            target_succ = set(digraph.successors(target))
            target_pred = set(digraph.predecessors(target))
            set_feature('len_source_successors', len(source_succ))
            set_feature('len_target_successors', len(target_succ))
            set_feature('len_source_predecessors', len(source_pred))
            set_feature('len_target_predecessors', len(target_pred))
            common_succ = len(source_succ.intersection(target_succ))
            common_pred = len(source_pred.intersection(source_pred))
            set_feature('common_successor_number', common_succ)
            set_feature('common_predecessor_number', common_pred)
            succ_union = source_succ.union(target_succ)
            pred_union = source_pred.union(target_pred)
            set_feature('jaccard_distance_between_successors',
                        common_succ / len(succ_union) 
                        if len(succ_union) != 0 
                        else 0)
            set_feature('jaccard_distance_between_predecessors',
                        common_pred / len(pred_union) 
                        if len(pred_union) != 0 
                        else 0)
            set_feature('source_pagerank', pagerank[source])
            set_feature('target_pagerank', pagerank[target])
            set_feature('source_authorities', auth[source])
            set_feature('target_authorities', auth[target])
            set_feature('shortest_path', nx.shortest_path_length(digraph, source, target)
                        if digraph.has_edge(source, target) 
                        else 0)
            set_feature('cluster_source', nx.clustering(graph, source))
            set_feature('cluster_target', nx.clustering(graph, target))
            pref_attch = nx.preferential_attachment(graph, [(source, target)])
            for u, v, p in pref_attch:
                set_feature('preference_attachment',p)
            aa_index = nx.adamic_adar_index(graph, [(source, target)])
            for u, v, p in aa_index:
                set_feature('adamic_adar_index',p)
            jcd_coe = nx.jaccard_coefficient(graph, [(source, target)])
            for u, v, p in jcd_coe:
                set_feature('jaccard_coefficient',p)
            reallo_index = nx.resource_allocation_index(graph, [(source, target)])
            for u, v, p in reallo_index:
                set_feature('resource_allocation_index',p)
        except:
            pass
        return features

    pos_feature, pos_label = feature_extraction(data_list[0], negative)
    neg_feature, neg_label = feature_extraction(data_list[1], negative)
    test_feature, _ = feature_extraction(data_list[2], negative)
    return pos_feature, pos_label, neg_feature, neg_label, test_feature
#print(pos_label)

pos_neg_test = [train_data, neg_train_data, test_data]
pos_input, pos_label, neg_input, neg_label, test_input = to_preprocess(pos_neg_test, True)
#print(pos_input)

input_df = pd.DataFrame(data=pos_input)
csv = input_df.to_csv(index=False, sep='\t')
with open('positive.csv', 'w') as file:
    file.write(csv)
    
input_df = pd.DataFrame(data=neg_input)
csv = input_df.to_csv(index=False, sep='\t')
with open('negative.csv', 'w') as file:
    file.write(csv)
    
input_df = pd.DataFrame(data=test_input)
csv = input_df.to_csv(index=False, sep='\t')
with open('test.csv', 'w') as file:
    file.write(csv)

4016
Instance processing over 100 out of 26937
Instance processing over 200 out of 26937
Instance processing over 300 out of 26937
Instance processing over 400 out of 26937
Instance processing over 500 out of 26937
Instance processing over 600 out of 26937
Instance processing over 700 out of 26937
Instance processing over 800 out of 26937
Instance processing over 900 out of 26937
Instance processing over 1000 out of 26937
Instance processing over 1100 out of 26937
Instance processing over 1200 out of 26937
Instance processing over 1300 out of 26937
Instance processing over 1400 out of 26937
Instance processing over 1500 out of 26937
Instance processing over 1600 out of 26937
Instance processing over 1700 out of 26937
Instance processing over 1800 out of 26937
Instance processing over 1900 out of 26937
Instance processing over 2000 out of 26937
Instance processing over 2100 out of 26937
Instance processing over 2200 out of 26937
Instance processing over 2300 out of 26937
Instance proces

In [ ]:



def get_feature(source, target):
    
    features = {}
    
    def set_feature(name, val):
        if name not in features:
            features[name] = val
    
    for d in feature_file:
    #print(d)
    #l = []
        for key in d.keys():
            if key.startswith('keyword'):
                set_feature('id', key[8:])
    #print(l)
    #di[d['id']] = l     
    return features

pos_feature, pos_label = feature_extraction(train_data, negative_instance = True)
#neg_feature, neg_label = feature_extraction(neg_train_data, negative_instance = False)
#test_feature, _ = feature_extraction(data_list[2], negative_instance)


In [ ]:
import json

f3 = open('/Users/enn/Desktop/sml/assignment1/data/nodes.json', 'r')
feature_file = json.load(f3)

di = {}
for d in feature_file:
    #print(d)
    l = []
    for key in d.keys():
        if key.startswith('keyword'):
            l.append(key[8:])
    #print(l)
    di[d['id']] = l
print(di)